# To do:

Structure data so that yelp and foursquare IDs are matched to menus

Combine foursquare and yelp menus

Get prices from Foursquare

In [1]:
import requests
import os
import importlib
import pandas as pd
import numpy as np
%matplotlib inline
import pdb
import json
import time
from Kibbeh.src import feature_extract_resto
from Kibbeh.src import API_io
import re
import matplotlib.pyplot as plt
import ast

In [ ]:
#working_dir = 'C:\\Users\\Chauncey\\Documents\\GitHub\\Kibbeh'
working_dir = 'C:\\Users\\Me\\Documents\\GitHub\\Kibbeh'
seattle_zips = pd.read_csv(working_dir + '\\seattle_zips.csv')['Zip'].values
portland_zips = pd.read_csv(working_dir + '\\portland_zips.csv')['Zip'].values
bay_area_zips = pd.read_csv(working_dir + '\\bay_area_zips.csv')['Zip'].values

In [3]:
# load foursquare restaurants and ratings
with open('fs_restaurant_list.json', 'r') as json_file:
    fs_top_restos = json.load( json_file)
len(fs_top_restos)

3081

In [4]:
# load foursquare menus
with open('fs_restaurant_menus.json', 'r') as json_file:
    fs_menus_rated = json.load( json_file)
len(fs_menus_rated)

332

In [5]:
# load yelp restaurants
with open('yelp_restaurant_list.json', 'r') as json_file:
    yelp_restos = json.load( json_file)
len(yelp_restos)

1000

In [6]:
# load from local json
with open('locu_restaurant_list.json', 'r') as json_file:
    locu_restos = json.load( json_file)
len(locu_restos)

1564

In [7]:
# load
with open('locu_menu_list.json', 'r') as json_file:
    locu_menus = json.load( json_file)
len(locu_menus)

399

# Scrape Yelp data

In [ ]:
from yelpapi import YelpAPI
with open(working_dir+ '\\yelp_api_key.txt', 'r') as api_file:
    api_key =  api_file.read()
    [CONSUMER_KEY, CONSUMER_SECRET, TOKEN, TOKEN_SECRET] = api_key.split('\n')
yelp_api = YelpAPI(CONSUMER_KEY, CONSUMER_SECRET, TOKEN, TOKEN_SECRET)

In [ ]:
yelp_restos = API_io.get_yelp_restaurants(yelp_api)
len(yelp_restos)

In [ ]:
# save
with open('yelp_restaurant_list.json', 'w') as json_file:
    json.dump(yelp_restos, json_file)

# Scrape Foursquare data
### Top restaurants in each zip code

In [9]:
import foursquare
with open(working_dir+ '\\foursquare_api_key.txt', 'r') as api_file:
    api_key =  api_file.read()
    [FOURSQUARE_ID, FOURSQUARE_SECRET] = api_key.split('\n')
foursquare_client = foursquare.Foursquare(FOURSQUARE_ID, FOURSQUARE_SECRET)

In [37]:
# get data from all seattle zip codes
# each request gets 250
fs_top_restos2 = API_io.get_fs_top_restos_zip(foursquare_client, bay_area_zips[100:150])
# this takes a while to run
len(fs_top_restos2)

Download data for 94025 zip code.
Download data for 94028 zip code.
Download data for 94027 zip code.
Download data for 94030 zip code.
Download data for 94553 zip code.
Download data for 94037 zip code.
Download data for 94038 zip code.
Download data for 94044 zip code.
Download data for 94060 zip code.
Download data for 94061 zip code.
Download data for 94541 zip code.
Download data for 97701 zip code.
Download data for 94002 zip code.
Download data for 94011 zip code.
Download data for 94005 zip code.
Download data for 94010 zip code.
Download data for 95002 zip code.
Download data for 95008 zip code.
Download data for 95013 zip code.
Download data for 95014 zip code.
Download data for 95020 zip code.
Download data for 94022 zip code.
Download data for 94024 zip code.
Download data for 95030 zip code.
Download data for 95032 zip code.
Download data for 95035 zip code.
Download data for 95037 zip code.
Download data for 95140 zip code.
Download data for 94040 zip code.
Download data 

2334

In [38]:
fs_top_restos.extend(fs_top_restos2)
len(fs_top_restos)

7205

In [39]:
# save
with open('bay_area_restaurant_list_zips1-150.json', 'w') as json_file:
    json.dump(fs_top_restos, json_file)

### Random restaurants in a latitude longitude grid
Most of these do not have ratings or menus

In [ ]:
# not tested after offloading to API_io.py
seattle_grid = [-122.38, -122.28, 47.55, 47.70]
fs_ll_restos = API_io.get_fs_restos_by_ll(seattle_grid, foursquare_client, grid_step = 0.1)

In [ ]:
len(fs_ll_restos)

### Get menus from foursquare

In [40]:
# get foursquare venues that have both menu and rating
# use set intersection to get intersection of keys
fs_has_menu_rated = [x['venue'] for x in fs_top_restos if set(('menu', 'rating')) <= x['venue'].keys()]
fs_id_list = [x['id'] for x in fs_has_menu_rated]
len(fs_id_list)

4591

In [41]:
fs_menus_rated = API_io.get_fs_menus(foursquare_client,fs_id_list)
len(fs_menus_rated) # there are less menus than you hope because some menus not availabe through API

KeyboardInterrupt: 

In [30]:
# save menus
with open('portland_restaurant_menus.json', 'w') as json_file:
    json.dump(fs_menus_rated, json_file)

# Load OpenMenu data
Denied on OpenMenu API request

# Load Locu data
### Set up api key and class

In [ ]:
from Kibbeh.src.locu_api import VenueApiClient, MenuItemApiClient
with open(working_dir+ '\\locu_api_key.txt', 'r') as api_file:
    api_key =  api_file.read()
    [LOCU_KEY] = api_key.split('\n')
locu_venue_client = VenueApiClient(LOCU_KEY)

### Get list of locu restaurants by zip code

In [ ]:
import itertools
def get_locu_by_zip(venue_client, zip_list):
    # a set of 25 restaurants with menus for each zip code in seattle
    list_of_lists =  list(map(lambda x: venue_client.search(category=['restaurant'], has_menu=True, postal_code=x)['objects'],
                              zip_list) )
    return list(itertools.chain(*list_of_lists))

#venues = venue_client.search(locality = 'Seattle', has_menu=True)

In [ ]:
locu_restos = get_locu_by_zip(venue_client, seattle_zips)
len(locu_restos)

In [ ]:
# save
with open('locu_restaurant_list.json', 'w') as json_file:
    json.dump(locu_restos, json_file)

# Scrape menus for restaurants not on Foursquare

In [10]:
fs_restos_rated = [x for x in fs_top_restos if 'rating' in x['venue'].keys()]
len(fs_restos_rated)

2289

In [38]:
importlib.reload(feature_extract_resto)
fs_resto_df = feature_extract_resto.feature_extract_fs_list(fs_restos_rated)
fs_resto_df.drop_duplicates(inplace=True)

In [14]:
fs_resto_df.sort('rating').tail(10)

,categories,checkinsCount,id,phone,rating,tier,tipCount,usersCount
name,,,,,,,,
John Howie Steak,Steakhouse,9201,4aa98a52f964a5202b5420e3,4254400880,9.4,4,110,4241
Kedai Makan,Malaysian,3871,51020084e4b0fb39a60c00bf,NaN,9.4,1,61,1381
The Walrus and the Carpenter,Seafood,5948,4c60a06213791b8d151f50af,2063959227,9.4,3,168,4332
Canlis,American,3861,44fdc137f964a52097381fe3,2062833313,9.4,4,90,2672
Mammoth,Sandwiches,665,5499f417498ef6258913ed4e,2069204126,9.4,1,15,392
Buddha Ruksa,Asian,2628,466ed01ef964a520ba471fe3,2069377676,9.5,2,46,1180
DERU Market,Sandwiches,819,4e28aeed81dcff8f987ebebb,4252980268,9.5,2,25,354
Bakery Nouveau,Bakery,3737,51755180e4b01f6b164ed9b2,2068586957,9.5,2,52,1498
Spinasse,Italian,3062,49e0420af964a5204b611fe3,2062517673,9.5,3,67,1992


In [40]:
from bs4 import BeautifulSoup

In [204]:
fs_resto_df.loc['Longhorn Barbecue']

categories                            BBQ
checkinsCount                         771
id               4b959d1bf964a52030ac34e3
phone                          2538049600
rating                                7.9
tier                                    2
tipCount                               14
usersCount                            467
Name: Longhorn Barbecue, dtype: object

In [257]:
[x for x in fs_restos_rated if x['venue']['id'] == 
'4b959d1bf964a52030ac34e3']

[{'reasons': {'count': 0,
   'items': [{'reasonName': 'globalInteractionReason',
     'summary': 'This spot is popular',
     'type': 'general'}]},
  'referralId': 'e-3-4b959d1bf964a52030ac34e3-9',
  'tips': [{'canonicalUrl': 'https://foursquare.com/item/4c142502a9c220a1de1e579d',
    'createdAt': 1276388610,
    'id': '4c142502a9c220a1de1e579d',
    'likes': {'count': 4, 'groups': [], 'summary': '4 likes'},
    'logView': True,
    'text': 'Have pulled pork sandwich with some cole slaw on top',
    'todo': {'count': 0},
    'type': 'user',
    'user': {'firstName': 'Carl',
     'gender': 'male',
     'id': '1598189',
     'lastName': 'Holzboog',
     'photo': {'prefix': 'https://irs0.4sqi.net/img/user/',
      'suffix': '/4ECB3IVEXAZC1JG4.jpg'}}}],
  'venue': {'allowMenuUrlEdit': True,
   'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/bbqalt_',
      'suffix': '.png'},
     'id': '4bf58dd8d48988d1df931735',
     'name': 'BBQ Joint',
     'pluralName': 

In [247]:
new_menus

In [263]:
new_menus = get_menus_for_list(fs_restos_rated )

> <ipython-input-262-908764db2cb4>(13)get_menu()
-> resto_homepage = requests.get(resto_url)
(Pdb) n
> <ipython-input-262-908764db2cb4>(14)get_menu()
-> resto_home_soup = BeautifulSoup(resto_homepage.text)
(Pdb) n


C:\Users\Chauncey\Anaconda3\lib\site-packages\bs4\__init__.py:166: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

To get rid of this warning, change this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


> <ipython-input-262-908764db2cb4>(16)get_menu()
-> resto_url, resto_home_soup = check_redirect(resto_url, resto_home_soup)
(Pdb) n
> <ipython-input-262-908764db2cb4>(19)get_menu()
-> menu_regex = re.compile('[mM]enu|[dD]inner')
(Pdb) n
> <ipython-input-262-908764db2cb4>(20)get_menu()
-> menu_elements = resto_home_soup.find_all('a', text = menu_regex)
(Pdb) n
> <ipython-input-262-908764db2cb4>(22)get_menu()
-> menu_link = menu_elements[0].get('href') # check whether link is full or short
(Pdb) n
> <ipython-input-262-908764db2cb4>(23)get_menu()
-> if len(menu_link) > 15:
(Pdb) n
> <ipython-input-262-908764db2cb4>(26)get_menu()
-> menu_url = resto_url + '/' + menu_link
(Pdb) menu_url
*** NameError: name 'menu_url' is not defined
(Pdb) menu_link
'Menu'
(Pdb) menu_link
'Menu'
(Pdb) n
> <ipython-input-262-908764db2cb4>(28)get_menu()
-> menu_html = requests.get(menu_url)
(Pdb) menu_url
'http://senor-taco.com/Menu'
(Pdb) n
> <ipython-input-262-908764db2cb4>(29)get_menu()
-> menu_soup = Beauti

IndexError: list index out of range

In [210]:
def get_menus_for_list(foursquare_restos):
    new_menus = []
    for cur_resto in foursquare_restos:
        if 'hasMenu' not in cur_resto['venue'] and 'url' in cur_resto['venue']:
            new_menus.append({'id':cur_resto['venue']['id'],
                              'menus':get_menu(cur_resto['venue']['url']) })

In [262]:
def check_redirect(original_url, original_soup):
    redirect_meta = original_soup.find("meta",attrs={"http-equiv":re.compile('refresh', flags=re.IGNORECASE)})
    if redirect_meta:
        wait,text=redirect_meta["content"].split(';')
        new_url=text[text.lower().find('url') + 4:]
        new_html = requests.get(new_url)
        return new_url, BeautifulSoup(new_html.text)
    return original_url, original_soup

def get_menu(resto_url):
    pdb.set_trace()
    
    resto_homepage = requests.get(resto_url)
    resto_home_soup = BeautifulSoup(resto_homepage.text)
    
    resto_url, resto_home_soup = check_redirect(resto_url, resto_home_soup)
    #resto_url = check_url(resto_url)
    
    menu_regex = re.compile('[mM]enu|[dD]inner') 
    menu_elements = resto_home_soup.find_all('a', text = menu_regex)
    
    menu_link = menu_elements[0].get('href') # check whether link is full or short
    if len(menu_link) > 15:
        menu_url = menu_link
    else:
        menu_url = resto_url + '/' + menu_link
    
    menu_html = requests.get(menu_url)
    menu_soup = BeautifulSoup(menu_html.text)
    
    # get rid of JavaScript
    script_var = menu_soup.find_all('script')
    for script in script_var:
        script.decompose()
        
    menu_text = ' '.join(list(menu_soup.stripped_strings))
    return menu_text